In [16]:
import os, sys, email, datetime, pprint, re, time, html, pickle
import numpy as np
import pandas as pd
import nltk
from nltk.metrics import *

pd.options.display.max_colwidth = 1000

In [2]:
from db import connect
engine = connect()
mail_df = pd.read_sql(sql='SELECT * FROM mail_3 ORDER BY docno ASC',con=engine, index_col=None)

In [3]:
### 自然言語処理
import spacy 
nlp = spacy.load("en_core_web_sm")
from spacy.pipeline import Sentencizer
sentencizer = Sentencizer()
nlp.add_pipe(sentencizer)

In [4]:
mail_df.head()

,docno,received,isoreceived,sent,isosent,name,email,subject,id,charset,inreplyto,expires,to,cc,body
0,lists-000-13503992,Fri Jul 14 12:48:10 2000,2000-07-14 16:48:10,"Fri, 14 Jul 2000 08:19:41 -0400 (EDT)",2000-07-14 12:19:41,Georgopoulos N,ngeorgopoulos@city.ac.uk,Bug report,E13D4Rs-0004TB-00@mailswitch,US-ASCII,None,-1,html-tidy@w3.org,None,text/enriched attachment: stored
1,lists-001-9920622,Sat Jul 14 07:24:02 2001,2001-07-14 11:24:02,"Sat, 14 Jul 2001 13:12:58 +0200",2001-07-14 11:12:58,Karl Ove Hufthammer,huftis@bigfoot.com,Re: entities in title tags,003b01c10c55$f0f62060$f3468ed5@huftis,iso-8859-1,000201c10c12$b894f900$4222e540@IanEvans,-1,html-tidy@w3.org,None,"----- Original Message ----- \nFrom: ""Ian M. Evans"" <ianevans@digitalhit.com>\nSent: Saturday, July 14, 2001 5:11 AM\nSubject: entities in title tags\n Just wondering if there's anything wrong with entities in title tags.\nNo.\n I've noticed that the numeric entity for the apostrophe ends up showing up\n in Google search results as:\nThis is a bug in Google.\n-- \nKarl Ove Hufthammer"
2,lists-001-9920622,Sat Jul 14 07:24:02 2001,2001-07-14 11:24:02,"Sat, 14 Jul 2001 13:12:58 +0200",2001-07-14 11:12:58,Karl Ove Hufthammer,huftis@bigfoot.com,Re: entities in title tags,003b01c10c55$f0f62060$f3468ed5@huftis,iso-8859-1,000201c10c12$b894f900$4222e540@IanEvans,-1,ianevans@digitalhit.com,None,"----- Original Message ----- \nFrom: ""Ian M. Evans"" <ianevans@digitalhit.com>\nSent: Saturday, July 14, 2001 5:11 AM\nSubject: entities in title tags\n Just wondering if there's anything wrong with entities in title tags.\nNo.\n I've noticed that the numeric entity for the apostrophe ends up showing up\n in Google search results as:\nThis is a bug in Google.\n-- \nKarl Ove Hufthammer"
3,lists-002-10134409,Tue Apr 29 10:23:48 2003,2003-04-29 14:23:48,"Tue, 29 Apr 2003 22:23:41 +0800",2003-04-29 14:23:41,lock mode,klcc70@hotmail.com,"Stock markets rise on positive earnings, consumer spending ConsumerReportsorg",Law10-F458juXym3SjK00016b82@hotmail.com,None,None,-1,html-tidy@w3.org,None,"Unbiased product Ratings from the experts at Product Ratings, product \nreviews, buying guides, product safety recalls and consumer information from \nthe experts at Consumer Union/Consumer Reports Magazine. Information, \nratings, and advice on products, services, and decisions. new car reviews, \nused car Consumer Reports Cars and Trucks: Stay informed with our latest new \ncar reviews, used car reviews, auto ratings, car safety reports and auto \nprice service. Provides car reviews, automobile safety information, car \nbuying guidance.\nhttp://www.geocities.com/webrobotic2/ConsumerReports.htm\nconsumer report\n12965 consumer report magazine\n11859 consumer report car\n9172 consumer report online\n7784 consumer report auto\n4280 consumer report vacuum cleaners\n3970 consumer report digital camera\n3764 consumer credit report\n3633 consumer report automobile\n2817 free consumer report\n2477 consumer report mattress\n2221 consumer report on health\n1978 consumer report magazine sub..."
4,lists-002-10233238,Sat May 17 03:24:47 2003,2003-05-17 07:24:47,"Wed, 14 May 2003 09:32:46 -0400 (EDT)",2003-05-14 13:32:46,David Van Cott,dtvc17@msn.com,Tag problem,BAY4-F144h4Jzgi1Z8200004355@hotmail.com,None,None,-1,html-tidy@w3.org,dsr@w3.org,I'm using html tidy at work to make the website more accessible and I'm \nhaving a little problem. Everything seems to work great execept for one \nthing. Tidy seems to be removing the <dl tags. This is causing the \nalignment of the webpages to be off. I searched through google newsgroups \nand the www.w3.org site and I couldnt find any questions or solutions \nregarding this. There is my configs...\n--clean yes\n--output-xml no\n--enclose-text yes\n--enclose-block-text yes\n--indent auto\n--alt-text GraphicImage\nOther then that problem I think this program is great. I have to make sure \nover 1000 html files have alt attributes inside the img tag. This make file \nmuch

In [11]:
pattern = re.compile('\W+')
check = re.compile('')
count = 0
rows = []

for value in mail_df.values:
    docno = value[0]
    uid = value[8]
    body = value[14]
    doc = nlp(body)

    for sent in doc.sents:
        sent_text = sent.text
        if len(sent_text.split()) <= 5: # for uninformative sentence
            pass
        elif re.search('google', sent_text, flags=re.IGNORECASE):
            rows.append([docno,uid,sent_text])
        else:
            pass

In [12]:
len(rows)

696

In [13]:
checklist = pd.DataFrame(rows,columns=['docno','uid','sentence'])
checklist.head()

,docno,uid,sentence
0,lists-001-9920622,003b01c10c55$f0f62060$f3468ed5@huftis,I've noticed that the numeric entity for the apostrophe ends up showing up\n in Google search results as:\n
1,lists-001-9920622,003b01c10c55$f0f62060$f3468ed5@huftis,This is a bug in Google.
2,lists-001-9920622,003b01c10c55$f0f62060$f3468ed5@huftis,I've noticed that the numeric entity for the apostrophe ends up showing up\n in Google search results as:\n
3,lists-001-9920622,003b01c10c55$f0f62060$f3468ed5@huftis,This is a bug in Google.
4,lists-002-10233238,BAY4-F144h4Jzgi1Z8200004355@hotmail.com,I searched through google newsgroups \nand the www.w3.org site and I couldnt find any questions or solutions \nregarding this.


In [14]:
checklist.to_csv('/Users/taroaso/myprojects/OpenIE/trec/output/3/eval3_checklist.csv')

In [22]:
with open('/Users/taroaso/myprojects/OpenIE/trec/output/3/openie_result.pickle', mode="rb") as f:
    openie = pickle.load(f)

In [23]:
rows = []
for uid, sents in openie.items():
    for sent in sents:
        if re.search('google', sent['sentence'], flags=re.IGNORECASE):
            rows.append([uid, sent['sentence']])

In [25]:
len(rows)

289

In [26]:
checklist = pd.DataFrame(rows,columns=['uid','sentence'])
checklist.head()

,uid,sentence
0,4084867E.9050502@kbc.net.au,"Now, from the point of view of accessibility, when a website's pages are indexed by Google, wouldn't it be easier for people with disabilities to use a seach engine they are already used to use?"
1,4084867E.9050502@kbc.net.au,"Should we actively promote the use of search engines like Google as local seach engine at any major website, if the pages are already indexed by Google?"
2,4084867E.9050502@kbc.net.au,"Whilst I think that Google has a great product, I do not think it in the best \ninterests of accessibility, or indeed of a stable infrastructure, to standardise \non a single, proprietary solution."
3,4084867E.9050502@kbc.net.au,"If Google should fail as a company or start charging excessively for the Site \nSearch service), if everyone were using it, the entire concept of Site Search \nwould drop into a vacuum."
4,4084867E.9050502@kbc.net.au,"ie: not \nall owned by Google), I might feel otherwise."


In [27]:
checklist.to_csv('/Users/taroaso/myprojects/OpenIE/trec/output/3/eval3_checklist.csv')